# Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# 연결 변수
AWS_ACCESS_KEY_ID ="AWS_SECRET_ACCESS_KEY = "AWS_WAREHOUSE_DIR = "s3a://donghee-s3-alldata/WareHouse"
AWS_DEFAULT_REGION = "ap-northeast-2"

# spark session 생성시 aws와 연동하기
def aws_connect_spark(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_WAREHOUSE_DIR):
    # 설정
    conf = (
        SparkConf()
        .setAppName("MY_APP") # replace with your desired name
        .set("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.4")
        .set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
        .set("spark.databricks.delta.properties.defaults.columnMapping.mode","name") # header 공백 및 특수문자 인식
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") # Deltalake로 Apache Spark 설정
        .set("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") # Deltalake로 Apache Spark 설정
        .set("spark.sql.parquet.columnarReaderBatchSize", 100) # ?
        .set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID)
        .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY)
        .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .set("spark.sql.warehouse.dir", AWS_WAREHOUSE_DIR)
        .set("spark.executor.memory", "8g") # 각 Spark worker의 memory 크기
        .set("spark.driver.memory", "2g") # Spark Driver의 크기
    )

    # spark 생성
    spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()
    
    return spark

In [2]:
spark = aws_connect_spark(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_WAREHOUSE_DIR)

spark

24/04/17 00:03:12 WARN Utils: Your hostname, donghee-MS-7C94 resolves to a loopback address: 127.0.1.1; using 192.168.75.133 instead (on interface enp42s0)
24/04/17 00:03:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/donghee/.ivy2/cache
The jars for the packages stored in: /home/donghee/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-40721933-5eee-4a5f-a353-48f1df07b709;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/donghee/work/deltalakeenv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 161ms :: artifacts dl 11ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-

# Data 가져오기

In [3]:
import pyspark.sql.functions as F

In [4]:
table_path = 's3a://donghee-s3-alldata/Nov_Delta'

csv_path = 's3a://donghee-s3-alldata/csv/2019-Nov.csv'

In [5]:
# s3 csv -> pyspark dataframe
Nov_df = spark.read.format("csv").option("header",True).load(csv_path)

Nov_df.show()

type(Nov_df)

+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 00:00:...|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:...|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:...|      view|  17302664|2053013553853497655|                NULL|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:...|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:...|      view|   1004775|2053013555631882655|elect

pyspark.sql.dataframe.DataFrame

In [6]:
Nov_df = Nov_df.withColumn("event_time", F.to_timestamp(F.col("event_time")))

In [7]:
Nov_df.show()

+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 09:00:00|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 09:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 09:00:01|      view|  17302664|2053013553853497655|                NULL|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 09:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 09:00:01|      view|   1004775|2053013555631882655|electronics.s

In [10]:
# Nov_df.write.format("delta").save("s3a://donghee-s3-alldata/Nov_Data")

----

In [6]:
# 해당 날짜만 추출
table_path = "s3a://donghee-s3-alldata/Nov_Data"

In [7]:
# 가진 날짜들 추출
date_list = spark.sql(F"SELECT DISTINCT DATE(event_time) FROM delta.`{table_path}`").sort("event_time").collect()

# 첫번째 날짜 확인
df = spark.sql(F"SELECT * FROM delta.`{table_path}` WHERE DATE(event_time) = '{date_list[0]['event_time']}'")

df.show()

24/04/16 20:43:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/16 20:43:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/16 20:44:00 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/04/16 20:44:00 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore donghee@127.0.1.1
24/04/16 20:44:00 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/04/16 20:44:00 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/04/16 20:44:00 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:44:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields

+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 09:00:00|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 09:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 09:00:01|      view|  17302664|2053013553853497655|                NULL|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 09:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 09:00:01|      view|   1004775|2053013555631882655|electronics.s

In [14]:
# 날짜별 행 데이터 개수 확인
for i in date_list:
    print(i, ' count =', spark.sql(F"SELECT * FROM delta.`{table_path}` WHERE DATE(event_time) = '{i['event_time']}'").count())

24/04/15 20:34:25 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


Row(event_time=datetime.date(2019, 11, 1))  count = 947011


24/04/15 20:34:26 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


Row(event_time=datetime.date(2019, 11, 2))  count = 1521765


24/04/15 20:34:26 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


...

Row(event_time=datetime.date(2019, 11, 29))  count = 1827111


24/04/15 20:34:43 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


Row(event_time=datetime.date(2019, 11, 30))  count = 1782859


24/04/15 20:34:44 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


Row(event_time=datetime.date(2019, 12, 1))  count = 588056


In [15]:
spark.sql(F"SELECT * FROM delta.`{table_path}`").printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



24/04/15 20:34:47 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


---
# 테이블 만들기

In [8]:
spark.sql(
"CREATE TABLE delta.`s3a://donghee-s3-alldata/Nov_Partition` (\
    event_time TIMESTAMP,\
    event_type STRING,\
    product_id STRING,\
    category_id STRING,\
    category_code STRING,\
    brand STRING,\
    price STRING,\
    user_id STRING,\
    user_session STRING,\
    date DATE\
) USING DELTA PARTITIONED BY (date)"
)

24/04/16 20:44:45 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


DataFrame[]

In [6]:
# 해당 날짜만 추출
Nov_partition = "s3a://donghee-s3-alldata/Nov_Partition"

In [10]:
for i in date_list:
    # 시스템 timestamp 수정...?
    df = spark.sql(F"SELECT * FROM delta.`{table_path}` WHERE DATE(event_time) = '{i['event_time']}'").withColumn("date", F.to_date(F.col("event_time").cast("timestamp")))

    df.write.mode("append")\
    .format("delta")\
    .save(Nov_partition)

    spark.sql(F"UPDATE delta.`{Nov_partition}` SET event_type = 'VIEW' WHERE DATE(event_time) = '{i['event_time']}' AND event_type = 'view'")
    spark.sql(F"UPDATE delta.`{Nov_partition}` SET event_type = 'PURCHASE' WHERE DATE(event_time) = '{i['event_time']}' AND event_type = 'purchase'")
    spark.sql(F"UPDATE delta.`{Nov_partition}` SET event_type = 'CART' WHERE DATE(event_time) = '{i['event_time']}' AND event_type = 'cart'")

24/04/16 20:45:33 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:45:39 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:45:45 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:45:50 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:45:56 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:46:02 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:46:09 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:46:15 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:46:21 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:46:28 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/16 20:46:35 WA

---
# Nov Partition

In [5]:
Partition_Compact_Table = "s3a://donghee-s3-alldata/Nov_Partition_Compaction"

In [18]:
for i in date_list:
    # 시스템 timestamp 수정...?
    df = spark.sql(F"SELECT * FROM delta.`{table_path}` WHERE DATE(event_time) = '{i['event_time']}'").withColumn("date", F.to_date(F.col("event_time").cast("timestamp")))

    df.write.mode("append")\
    .format("delta")\
    .save(Partition_Compact_Table)

    spark.sql(F"UPDATE delta.`{Partition_Compact_Table}` SET event_type = 'VIEW' WHERE DATE(event_time) = '{i['event_time']}' AND event_type = 'view'")
    spark.sql(F"UPDATE delta.`{Partition_Compact_Table}` SET event_type = 'PURCHASE' WHERE DATE(event_time) = '{i['event_time']}' AND event_type = 'purchase'")
    spark.sql(F"UPDATE delta.`{Partition_Compact_Table}` SET event_type = 'CART' WHERE DATE(event_time) = '{i['event_time']}' AND event_type = 'cart'")

24/04/15 20:35:12 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:35:19 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:35:25 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:35:31 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:35:37 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:35:43 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:35:49 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:35:55 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:36:01 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:36:08 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
24/04/15 20:36:16 WA

In [26]:
spark.sql(F"SELECT * FROM delta.`{Partition_Compact_Table}`").show()

24/04/16 20:25:34 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


+-------------------+----------+----------+-------------------+--------------------+------------+-------+---------+--------------------+----------+
|         event_time|event_type|product_id|        category_id|       category_code|       brand|  price|  user_id|        user_session|      date|
+-------------------+----------+----------+-------------------+--------------------+------------+-------+---------+--------------------+----------+
|2019-11-04 16:28:59|      VIEW|  32900143|2055156924407612189|     accessories.bag|    flambeau|  25.70|517005110|d25c3e55-2615-47c...|2019-11-04|
|2019-11-04 16:28:59|      VIEW|   1004767|2053013555631882655|electronics.smart...|     samsung| 241.86|567196140|7349f7a1-e89a-417...|2019-11-04|
|2019-11-04 16:28:59|      VIEW|   1802017|2053013554415534427|electronics.video.tv|          lg|1338.44|561121624|bee89ce0-6171-4f8...|2019-11-04|
|2019-11-04 16:28:59|      VIEW|   3601605|2053013563810775923|appliances.kitche...|          lg| 372.22|5671328

In [10]:
Nov_partition = "s3a://donghee-s3-alldata/Nov_Partition"

spark.sql(F"SELECT * FROM delta.`{Nov_partition}`").show()

24/04/17 00:12:49 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


+-------------------+----------+----------+-------------------+--------------------+---------+------+---------+--------------------+----------+
|         event_time|event_type|product_id|        category_id|       category_code|    brand| price|  user_id|        user_session|      date|
+-------------------+----------+----------+-------------------+--------------------+---------+------+---------+--------------------+----------+
|2019-11-11 19:52:34|      VIEW|   1004739|2053013555631882655|electronics.smart...|   xiaomi|203.17|558999650|8305ef9e-7496-402...|2019-11-11|
|2019-11-11 19:52:34|      VIEW|   5100816|2053013553375346967|                NULL|   xiaomi| 30.85|539233506|7354e964-adb4-6a5...|2019-11-11|
|2019-11-11 19:52:34|      VIEW|  26205358|2053013563693335403|                NULL|     NULL|198.46|512815726|a606546a-283c-47e...|2019-11-11|
|2019-11-11 19:52:34|      VIEW|   5301741|2053013563173241677|                NULL|    vitek| 16.19|523688455|8e0df0c3-cd14-451...|2019

---

# Compaction Data

In [43]:
from delta import *

deltaTable = DeltaTable.forPath(spark, Partition_Compact_Table)

for i in date_list:
    deltaTable.optimize().where(F"date = '{i['event_time']}'").executeCompaction()

---
# VACUUM TEST

In [13]:
# NOT COMPACTION -> VACUUM
spark.sql(F"VACUUM delta.`{Nov_partition}` RETAIN 0 HOURS DRY RUN")

24/04/16 21:05:01 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


Found 284 files (7503052942 bytes) and directories in a total of 32 directories that are safe to delete.


DataFrame[path: string]

In [ ]:
spark.sql(F"VACUUM delta.`{Nov_partition}` RETAIN 0 HOURS")

In [12]:
# COMPACTION -> VACUUM
spark.sql(F"VACUUM delta.`{Partition_Compact_Table}` RETAIN 0 HOURS DRY RUN")

24/04/16 19:07:33 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


Found 381 files (10165354783 bytes) and directories in a total of 32 directories that are safe to delete.


DataFrame[path: string]

In [13]:
spark.sql(F"VACUUM delta.`{Partition_Compact_Table}` RETAIN 0 HOURS")

24/04/16 19:49:46 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


Deleted 381 files and directories in a total of 32 directories.


DataFrame[path: string]

---
# QUERY

In [15]:
%%time

# NOT COMPACTION NOT VACUUM
spark.sql(F"SELECT date, event_type, SUM(price) FROM delta.`{Nov_partition}` GROUP BY date, event_type ORDER BY date, event_type").show()

24/04/17 00:13:57 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


+----------+----------+--------------------+
|      date|event_type|          sum(price)|
+----------+----------+--------------------+
|2019-11-01|      CART|   5006892.930000016|
|2019-11-01|  PURCHASE|   5301294.389999897|
|2019-11-01|      VIEW|2.6649098207999858E8|
|2019-11-02|      CART|   6591873.659999869|
|2019-11-02|  PURCHASE|  6498311.9399999315|
|2019-11-02|      VIEW|4.3119719518004555E8|
|2019-11-03|      CART|   6711994.019999986|
|2019-11-03|  PURCHASE|    6581772.13999999|
|2019-11-03|      VIEW| 4.638097055400138E8|
|2019-11-04|      CART|   7583235.520000016|
|2019-11-04|  PURCHASE|   7658979.530000003|
|2019-11-04|      VIEW|4.8062358463018155E8|
|2019-11-05|      CART|   6970336.040000062|
|2019-11-05|  PURCHASE|   7435775.660000044|
|2019-11-05|      VIEW| 4.966589007000288E8|
|2019-11-06|      CART|   6828105.470000026|
|2019-11-06|  PURCHASE|   7421478.290000043|
|2019-11-06|      VIEW| 4.688426716599125E8|
|2019-11-07|      CART|   6899707.409999935|
|2019-11-0

In [16]:
%%time

# COMPACTION VACUUM
spark.sql(F"SELECT date, event_type, SUM(price) FROM delta.`{Partition_Compact_Table}` GROUP BY date, event_type ORDER BY date, event_type").show()

24/04/17 00:14:10 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


+----------+----------+--------------------+
|      date|event_type|          sum(price)|
+----------+----------+--------------------+
|2019-11-01|      CART|   5006892.930000016|
|2019-11-01|  PURCHASE|   5301294.389999897|
|2019-11-01|      VIEW|2.6649098207999858E8|
|2019-11-02|      CART|   6591873.659999801|
|2019-11-02|  PURCHASE|   6498311.939999864|
|2019-11-02|      VIEW| 4.311971951795948E8|
|2019-11-03|      CART|   6711994.020000118|
|2019-11-03|  PURCHASE|   6581772.140000085|
|2019-11-03|      VIEW| 4.638097055401653E8|
|2019-11-04|      CART|   7583235.520000072|
|2019-11-04|  PURCHASE|   7658979.530000119|
|2019-11-04|      VIEW|4.8062358463029426E8|
|2019-11-05|      CART|  6970336.0399998585|
|2019-11-05|  PURCHASE|   7435775.659999984|
|2019-11-05|      VIEW| 4.966589007002334E8|
|2019-11-06|      CART|   6828105.470000158|
|2019-11-06|  PURCHASE|   7421478.290000171|
|2019-11-06|      VIEW| 4.688426716602296E8|
|2019-11-07|      CART|   6899707.410000093|
|2019-11-0

In [13]:
spark.sql(F"DESCRIBE HISTORY delta.`{Partition_Compact_Table}`").show()

24/04/17 00:13:29 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|    154|2024-04-16 17:46:15|  NULL|    NULL| OPTIMIZE|{predicate -> ["(...|NULL|    NULL|     NULL|        153|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Apache-Spark/3.5....|
|    153|2024-04-16 17:46:10|  NULL|    NULL| OPTIMIZE|{predicate -> ["(...|NULL|    NULL|     NULL|        152|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Apache-Spark/3.5....|
